In [ ]:
import pyxdf
import soundfile as sf
import numpy as np
import mne
import os
from autoreject import AutoReject
%matplotlib qt

In [ ]:
def segment_modalities_optimized(marker_timestamps, modality_timestamps, modality_data):
    """
    Optimized segmentation of data for a modality based on marker timestamps, accommodating different sampling rates.

    :param marker_timestamps: Timestamps of markers.
    :param modality_timestamps: Timestamps of the modality data.
    :param modality_data: Data of the modality to be segmented.
    :return: A list of data segments for the modality.
    """
    # Convert to numpy arrays for efficient computation
    marker_timestamps = np.array(marker_timestamps)
    modality_timestamps = np.array(modality_timestamps)

    # Find insertion points for each marker in the modality timestamps
    insert_points = np.searchsorted(modality_timestamps, marker_timestamps)
    segments = []
    for i in range(len(insert_points) - 1):
        # Extract and store the segment
        start_index = insert_points[i]
        end_index = insert_points[i + 1]
        segment = modality_data[start_index:end_index]
        segments.append(segment)

    # Handle the last segment, from the last marker to the end of the data stream
    if insert_points[-1] < len(modality_data):
        last_segment = modality_data[insert_points[-1]:]
        segments.append(last_segment)
    else:
        # If the last marker is exactly at or beyond the end of the data, append an empty segment
        segments.append([])

    return segments

In [ ]:
data, header = pyxdf.load_xdf('/Users/anarghya/Developer/eeg_data/multimodal-speech-eeg/dku/sub-8/part-1/sub-08_task-words_run-001.xdf',
                              select_streams=[{'type': 'EEG'}, {'type': 'Markers'}])

In [ ]:
data_types = ['EEG', 'Markers', 'Audio']
assert len(data) == 3
# check whether the data has 3 streams of type 'Markers', 'EEG' and 'Audio' index can be different
assert all([d['info']['type'][0] in data_types for d in data])

In [ ]:
marker_stream = [d for d in data if d['info']['type'][0] == 'Markers'][0]
eeg_stream = [d for d in data if d['info']['type'][0] == 'EEG'][0]
# audio_stream = [d for d in data if d['info']['type'][0] == 'Audio'][0]

In [ ]:
# Part 1 
stim_file = os.path.join('stimuli','stimuli_words.txt')
labels = open(stim_file).read().splitlines()

prefix = ['rest', 'stim',  'I', 'S']
marker_names = np.array(marker_stream['time_series']).squeeze()
marker_dict = {p: i for i, p in enumerate(np.unique(marker_names))}
id_binding = {v: k for k, v in marker_dict.items()}
category_mapping = {p: [v for k, v in marker_dict.items() if k.startswith(p)] for p in prefix}
marker_timestamps = np.array(marker_stream['time_stamps'])
modality_timestamps = np.array(eeg_stream['time_stamps'])
insert_points = np.searchsorted(modality_timestamps, marker_timestamps)

def filter_markers(marker_dict, prefix='stim'):
    """
    Filter markers based on the prefix.
    """
    return {k: v for k, v in marker_dict.items() if k.startswith(prefix)}

In [ ]:
ch_labels = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4']
sampling_rate = int(eeg_stream['info']['nominal_srate'][0])
info = mne.create_info(ch_names=ch_labels, sfreq=sampling_rate, ch_types='eeg')
data = eeg_stream["time_series"].T
raw = mne.io.RawArray(data, info)
raw.set_montage('standard_1020')
# raw.drop_channels(['Fp1','P8'])
# Apply bandpass filter
raw = raw.filter(l_freq=0.3, h_freq=60)
# # Apply notch filter
raw = raw.notch_filter(freqs=50)

label_id_func = np.vectorize(marker_dict.get)
events = np.zeros((len(insert_points), 3), dtype=int)
events[:, 0] = insert_points
events[:, 2] = label_id_func(marker_names)
annot = mne.annotations_from_events(events, raw.info['sfreq'], id_binding)
raw.set_annotations(annot)

# imagine = mne.pick_events(events, include=category_mapping['S'])
# epoch = mne.Epochs(raw, imagine, event_id=filter_markers(marker_dict, prefix='S'), tmin=-0.5, tmax=1.5, baseline=None, preload=True)

In [ ]:
raw.plot(scalings='auto', events=events, event_id=marker_dict, duration=10, n_channels=16, remove_dc=False)

In [ ]:
speak = mne.pick_events(events, include=category_mapping['S'])
imagine = mne.pick_events(events, include=category_mapping['I'])
# print(speak)
s_epochs = mne.Epochs(raw, speak, event_id=filter_markers(marker_dict, prefix='S'), tmin=-0.2, tmax=2, preload=True)
i_epochs = mne.Epochs(raw, imagine, event_id=filter_markers(marker_dict, prefix='I'), tmin=-0.2, tmax=2, preload=True)

In [ ]:
s_epochs.plot_image(combine='mean')

In [ ]:
i_epochs.plot_image(combine='mean')

In [ ]:
s_epochs.compute_psd(fmin=2.0, fmax=40.0).plot(picks="data", exclude="bads")

In [ ]:
i_epochs.compute_psd(fmin=2.0, fmax=40.0).plot(picks="data", exclude="bads")

In [ ]:
freqs = np.logspace(*np.log10([1, 40]), num=8)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power = mne.time_frequency.tfr_multitaper(s_epochs, freqs=freqs, n_cycles=n_cycles, return_itc=False, average=True)

In [ ]:
power.plot(
    baseline=(None, 0), mode="mean", tmin=-0.5, tmax=2, 
)

In [ ]:
ar = AutoReject()
epochs_clean = ar.fit_transform(epochs)

In [ ]:
epochs_clean.plot(scalings='auto')

In [ ]:
epochs_clean.plot_image(combine='mean')

In [ ]:
epochs_clean.compute_psd().plot_topomap()

In [ ]:
evoked = epochs_clean.average()

In [ ]:
evoked.plot_joint()

In [ ]:
s = segment_modalities_optimized(marker_stream['time_stamps'], audio_stream['time_stamps'], audio_stream['time_series'].squeeze())

In [ ]:
import pandas as pd

In [ ]:
s[6].shape[0]

In [ ]:
name = 'S-Zebra'
idx = np.where(marker_names == name)[0][0]
print(idx)

In [ ]:
def write_audio_data(audio_data_buffer, out_path='audio.wav'):
    audio_fs = 44100
    sf.write(out_path, audio_data_buffer, audio_fs)

In [ ]:
write_audio_data(s[idx], f'{name}.wav')